# Time Series Prediction with LSTM Using PyTorch

## Download Dataset

In [23]:
# General
import numpy as np
import time
# Loading data
import pandas as pd
from io import StringIO, BytesIO
from zipfile import ZipFile
import urllib.request
import os

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
#import pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

#Evaluation
from sklearn.metrics import mean_squared_error
from math import sqrt

## Load dataset

In [7]:
def url2pd(link):
    with ZipFile(link) as my_zip_file:
        for contained_file in my_zip_file.namelist():
            fzip=my_zip_file.open(contained_file)
            data=fzip.read()
    s=str(data,'utf-8')
    data = StringIO(s) 
    print('Done loading a dataset!')
    return pd.read_csv(data)

In [8]:
filenames = os.listdir('./Data')
url_1 = './Data/' + filenames[0]
df = url2pd(url_1)
df.info()
df.head(3)

Done loading a dataset!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Vehicle_ID    1048575 non-null  int64  
 1   Frame_ID      1048575 non-null  int64  
 2   Total_Frames  1048575 non-null  int64  
 3   Global_Time   1048575 non-null  int64  
 4   Local_X       1048575 non-null  float64
 5   Local_Y       1048575 non-null  float64
 6   Global_X      1048575 non-null  float64
 7   Global_Y      1048575 non-null  float64
 8   v_Length      1048575 non-null  float64
 9   v_Width       1048575 non-null  float64
 10  v_Class       1048575 non-null  int64  
 11  v_Vel         1048575 non-null  float64
 12  v_Acc         1048575 non-null  float64
 13  Lane_ID       1048575 non-null  int64  
 14  Preceeding    1048575 non-null  int64  
 15  Following     1048575 non-null  int64  
 16  Space_Hdwy    1048575 non-null  float64
 17  Tim

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID,Preceeding,Following,Space_Hdwy,Time_Hdwy
0,2,13,437,1118846980200,16.467196,35.380427,6451137.641,1873344.962,14.5,4.9,2,40.000000,0.000000,2,0,0,0.0,0.0
1,2,14,437,1118846980300,16.446594,39.381608,6451140.329,1873342.000,14.5,4.9,2,40.012349,0.123485,2,0,0,0.0,0.0
2,2,15,437,1118846980400,16.425991,43.381541,6451143.018,1873339.038,14.5,4.9,2,39.999855,-0.124939,2,0,0,0.0,0.0


In [9]:
# Filter time step
print(df.shape)
df = df.iloc[::2,:].copy()
df.reset_index(drop=True, inplace=True)
print('After filtering:', df.shape)
#  keep only columns that are useful for now
kept_cols = ['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Local_X','Local_Y','v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID']
df = df[kept_cols]
df.head(3)

(1048575, 18)
After filtering: (524288, 18)


,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
0,2,13,437,16.467196,35.380427,14.5,4.9,2,40.000000,0.000000,2
1,2,15,437,16.425991,43.381541,14.5,4.9,2,39.999855,-0.124939,2
2,2,17,437,16.384804,51.379881,14.5,4.9,2,39.991544,-0.013759,2


In [19]:
subdf = df[df.Vehicle_ID==2582].copy()
subdf.head()

,Vehicle_ID,Frame_ID,Total_Frames,Local_X,Local_Y,v_Length,v_Width,v_Class,v_Vel,v_Acc,Lane_ID
466051,2582,7263,1010,4.725434,36.935259,18.5,6.9,2,45.030000,0.000000,1
466052,2582,7265,1010,4.668727,45.875960,18.5,6.9,2,44.918464,4.280980,1
466053,2582,7267,1010,4.627103,54.924527,18.5,6.9,2,45.298705,1.107846,1
466054,2582,7269,1010,4.591890,63.945685,18.5,6.9,2,44.961476,-2.893200,1
466055,2582,7271,1010,4.543895,72.797674,18.5,6.9,2,44.002607,-5.160000,1


## Dataloading

In [21]:
training_set = subdf.Local_X.values.reshape(subdf.shape[0],1)
training_set.shape

(505, 1)

In [24]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

## Model

In [25]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

## Training

In [41]:
trainX.to(device)

tensor([[[0.0146],
         [0.0111],
         [0.0086],
         [0.0065]],

        [[0.0111],
         [0.0086],
         [0.0065],
         [0.0036]],

        [[0.0086],
         [0.0065],
         [0.0036],
         [0.0009]],

        ...,

        [[0.6998],
         [0.6995],
         [0.6999],
         [0.7004]],

        [[0.6995],
         [0.6999],
         [0.7004],
         [0.7006]],

        [[0.6999],
         [0.7004],
         [0.7006],
         [0.7005]]], device='cuda:0')

In [43]:
%time
num_epochs = 50
learning_rate = 0.001

input_size = 1
hidden_size = 100
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lstm.to(device)
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model

for epoch in range(num_epochs):
    trainX.to(device)
    outputs = lstm(trainX.cuda())
    
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 10 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

Wall time: 0 ns


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu

## Testing for Airplane Passengers Dataset

In [34]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)


In [35]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [36]:
rms = [sqrt(mean_squared_error(dataY_plot, data_predict))]
rms

[2.0545046184652205]